In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Geeting Data + Head

income_statement = pd.read_csv('sp500_income_statements.csv')
balance_sheet = pd.read_csv('sp500_balance_sheet.csv')
cash_flow = pd.read_csv('sp500_cash_flow.csv')
earnings = pd.read_csv('sp500_earnings.csv')
stock_prices = pd.read_csv('sp500_monthly_data.csv')
sector_info = pd.read_csv('sp500_tickers.csv')

# Merge everything, but only keep netIncome from the income statement
financial_data = (
    balance_sheet.merge(cash_flow.drop(columns=["netIncome"], errors="ignore"), on=["Ticker", "fiscalDateEnding"], how="inner")
    .merge(income_statement, on=["Ticker", "fiscalDateEnding"], how="inner")
    .merge(earnings, on=["Ticker", "fiscalDateEnding"], how="inner")
    .merge(stock_prices, left_on=["Ticker", "fiscalDateEnding"], right_on=["Ticker", "Date"], how="inner")
    .merge(sector_info, on="Ticker", how="left")
)

# Drop redundant Date column
financial_data.drop(columns=["Date"], inplace=True, errors="ignore")
print(financial_data.head())
# Combined datase

  fiscalDateEnding reportedCurrency_x   totalAssets  totalCurrentAssets  \
0       2024-12-31                USD  3.986800e+10        1.588400e+10   
1       2024-09-30                USD  4.087500e+10        1.629900e+10   
2       2023-06-30                USD  4.888000e+10        1.575400e+10   
3       2023-03-31                USD  4.688600e+10        1.496300e+10   
4       2022-09-30                USD  4.617600e+10        1.489500e+10   

   cashAndCashEquivalentsAtCarryingValue  cashAndShortTermInvestments  \
0                           5.600000e+09                 5.600000e+09   
1                           6.050000e+09                 6.050000e+09   
2                           4.258000e+09                 4.258000e+09   
3                           3.824000e+09                 3.824000e+09   
4                           3.404000e+09                 3.404000e+09   

      inventory  currentNetReceivables  totalNonCurrentAssets  \
0  3.698000e+09           3.194000e+09       

In [4]:
numeric_cols = financial_data.select_dtypes(include=["number"]).columns

financial_data[numeric_cols] = financial_data[numeric_cols].fillna(financial_data[numeric_cols].median())

financial_data.sort_values(by=["Ticker", "fiscalDateEnding"], inplace=True)
financial_data.ffill(inplace=True)

financial_data.dropna(axis=1, thresh=len(financial_data) * 0.5, inplace=True)

In [5]:
# Feature Engineering

financial_data["currentRatio"] = financial_data["totalCurrentAssets"] / financial_data["totalCurrentLiabilities"]
financial_data["debtToEquity"] = financial_data["totalLiabilities"] / financial_data["totalShareholderEquity"]
financial_data["netProfitMargin"] = financial_data["netIncome"] / financial_data["totalRevenue"]
financial_data["operatingCashFlowMargin"] = financial_data["operatingCashflow"] / financial_data["totalRevenue"]


In [6]:
# Liquidity Ratios
financial_data["quickRatio"] = (financial_data["totalCurrentAssets"] - financial_data["inventory"]) / financial_data["totalCurrentLiabilities"]

# Profitability Ratios
financial_data["returnOnAssets"] = financial_data["netIncome"] / financial_data["totalAssets"]
financial_data["returnOnEquity"] = financial_data["netIncome"] / financial_data["totalShareholderEquity"]

# Efficiency Ratios
financial_data["assetTurnover"] = financial_data["totalRevenue"] / financial_data["totalAssets"]

In [7]:
# Sort data before calculating returns
financial_data.sort_values(by=["Ticker", "fiscalDateEnding"], inplace=True)

# 1-month stock return
financial_data["monthly_return"] = financial_data.groupby("Ticker")["Close"].pct_change()

# Rolling 6-month volatility
financial_data["volatility_6m"] = financial_data.groupby("Ticker")["Close"].rolling(window=6).std().reset_index(0, drop=True)

In [8]:
# Replace infinities with NaN
financial_data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Select only numeric columns
numeric_cols = financial_data.select_dtypes(include=["number"]).columns

# Fill missing values with median (only for numeric columns)
financial_data[numeric_cols] = financial_data[numeric_cols].fillna(financial_data[numeric_cols].median())

In [1]:
# Standardization

from sklearn.preprocessing import StandardScaler

# Exclude non-numeric columns
num_cols = financial_data.select_dtypes(include=["number"]).columns

# Standardize numerical features only
scaler = StandardScaler()
financial_data[num_cols] = scaler.fit_transform(financial_data[num_cols])


NameError: name 'financial_data' is not defined

In [3]:
print(financial_data.isnull().sum())

fiscalDateEnding                         0
reportedCurrency_x                       2
totalAssets                              1
totalCurrentAssets                       0
cashAndCashEquivalentsAtCarryingValue    3
                                        ..
High                                     0
Low                                      0
Close                                    0
Volume                                   0
Sector                                   0
Length: 103, dtype: int64


In [35]:
financial_data.to_csv("normalized_financial_data.csv", index=False)

In [36]:
# Load the saved file to verify
df_check = pd.read_csv("normalized_financial_data.csv")
print(df_check.head())  # Check first few rows

  fiscalDateEnding reportedCurrency_x  totalAssets  totalCurrentAssets  \
0       2009-07-31                USD    -0.219533           -0.269218   
1       2010-04-30                USD    -0.215813           -0.195698   
2       2011-01-31                USD    -0.214949           -0.224793   
3       2011-10-31                USD    -0.211793           -0.199433   
4       2012-01-31                USD    -0.211662           -0.195620   

   cashAndCashEquivalentsAtCarryingValue  cashAndShortTermInvestments  \
0                              -0.141489                    -0.174209   
1                              -0.101249                    -0.140200   
2                              -0.101525                    -0.140258   
3                              -0.070871                    -0.114648   
4                              -0.066216                    -0.110684   

   inventory  currentNetReceivables  totalNonCurrentAssets  \
0  -0.004837              -0.378137              -0.35

In [11]:
financial_data.to_csv("financial_data.csv", index=False)

In [12]:
print(financial_data.columns.tolist())

['fiscalDateEnding', 'reportedCurrency_x', 'totalAssets', 'totalCurrentAssets', 'cashAndCashEquivalentsAtCarryingValue', 'cashAndShortTermInvestments', 'inventory', 'currentNetReceivables', 'totalNonCurrentAssets', 'propertyPlantEquipment', 'accumulatedDepreciationAmortizationPPE', 'intangibleAssets', 'intangibleAssetsExcludingGoodwill', 'goodwill', 'investments', 'longTermInvestments', 'shortTermInvestments', 'otherCurrentAssets', 'otherNonCurrentAssets', 'totalLiabilities', 'totalCurrentLiabilities', 'currentAccountsPayable', 'deferredRevenue', 'currentDebt', 'shortTermDebt', 'totalNonCurrentLiabilities', 'capitalLeaseObligations', 'longTermDebt', 'currentLongTermDebt', 'longTermDebtNoncurrent', 'shortLongTermDebtTotal', 'otherCurrentLiabilities', 'otherNonCurrentLiabilities', 'totalShareholderEquity', 'treasuryStock', 'retainedEarnings', 'commonStock', 'commonStockSharesOutstanding', 'Ticker', 'reportedCurrency_y', 'operatingCashflow', 'paymentsForOperatingActivities', 'proceedsFrom